In [ ]:
!pip install geopy

In [ ]:
!pip install haversine

In [ ]:
# 0206 수정
import pandas as pd
import numpy as np
from tqdm import tqdm
from geopy.geocoders import Nominatim
from haversine import haversine
import joblib

In [ ]:
census = pd.read_csv('/ml/algorithmlabs.csv/crawldata/20230126/HUIS.JUNGDO_CENSUS.csv')
edu = pd.read_csv('/ml/algorithmlabs.csv/crawldata/20230126/HUIS.JUNGDO_EDU.csv')
hakj = pd.read_csv('/ml/algorithmlabs.csv/crawldata/20230126/HUIS.JUNGDO_HAKJ.csv')
ipsi = pd.read_csv('/ml/algorithmlabs.csv/crawldata/20230126/HUIS.JUNGDO_IPSI.csv')
name = pd.read_csv('/ml/temp/feature_korean_name.csv')

In [ ]:
# 0206 추가
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [ ]:
# 0206 추가
lat = []
long = []
for i in range(ipsi['IPSI_SCHOOL_ADR'].nunique()) : 
    place = geocoding(ipsi['IPSI_SCHOOL_ADR'].unique().tolist()[i])
    lat.append(place[0])
    long.append(place[1])

In [ ]:
# 0206 추가
distance = []
for i in range(ipsi['IPSI_SCHOOL_ADR'].nunique()):
    start =(float(lat[i]), float(long[i]))
    goal = (float(geocoding('강원 춘천시')[0]), float(geocoding('강원 춘천시')[1]))
    distance.append(haversine(start, goal))

In [ ]:
# 0206 추가
place = ipsi['IPSI_SCHOOL_ADR'].unique().tolist()
diff_distance = []
for i in range(len(ipsi)) : 
    diff_distance.append(distance[place.index(ipsi['IPSI_SCHOOL_ADR'][i])])

ipsi['DIFF_DISTANCE'] = diff_distance

In [ ]:
temp_data = hakj.sort_values(by=['HAKJ_HAKBUN', 'YYHAKGI'], ascending=[True, True])
temp_data = temp_data.reset_index(drop=True)

In [ ]:
HUHAK_list = ['입대휴학', '일반휴학', '휴학', '창업휴학', '육아휴학']
cnt = 0 
seq_list = [0]

for i in range(1, len(temp_data)) : 
    if temp_data['HAKJ_HAKBUN'][i] == temp_data['HAKJ_HAKBUN'][i-1] : 
        if temp_data['BD_GBN_NAME'][i] in HUHAK_list : 
            cnt += 1
            seq_list.append(cnt)
        else : 
            cnt = 0
            seq_list.append(cnt)
    else : 
        cnt = 0
        if temp_data['BD_GBN_NAME'][i] in HUHAK_list : 
            cnt += 1
            seq_list.append(cnt)
        else : 
            seq_list.append(cnt)

In [ ]:
temp_data['seq'] = seq_list
# 시점에 따른 Labeling 작업 진행
semester_list = temp_data['YYHAKGI'].unique().tolist()
dropout_list = []
seq_drop_list = []
# temp_list = [] 

for i in range(len(temp_data)) : 
    if temp_data['YYHAKGI'][i] != 20222 : 
        next_semester = semester_list[semester_list.index(temp_data['YYHAKGI'][i])+1]
        temp_frame = temp_data.loc[(temp_data['YYHAKGI']==next_semester) & (temp_data['HAKJ_HAKBUN'] == temp_data['HAKJ_HAKBUN'][i])].reset_index(drop=True)
        if len(temp_frame) == 0 : 
            dropout_list.append("NoData")
            seq_drop_list.append("NOData")
        else : 
            dropout_list.append(temp_frame['BD_GBN_NAME'][0])
            seq_drop_list.append(temp_frame['seq'][0])
    else : 
        dropout_list.append("NoData")
        seq_drop_list.append("NoData")

In [ ]:
temp_data['HAKJ'] = dropout_list
temp_data['seq_la'] = seq_drop_list

In [ ]:
# 0206 수정
raw_data = temp_data.copy()
ipsi_list = []
none_list = []
for i in range(len(ipsi.columns)) : 
    none_list.append("NoData")

for i in range(len(raw_data)) : 
    if raw_data['HAKJ_HAKBUN'][i] in ipsi['IPSI_HAKSA_HAKBUN'].unique().tolist() : 
        ipsi_list.append(ipsi.loc[ipsi['IPSI_HAKSA_HAKBUN'] == raw_data['HAKJ_HAKBUN'][i]].reset_index(drop=True).values[0].tolist())
    else : 
        ipsi_list.append(none_list)

In [ ]:
# 0208 수정
raw_data[ipsi.columns.tolist()] = ipsi_list
en_data = raw_data.loc[raw_data['IPSI_PAN_NAME']!='NoData'].reset_index(drop=True)
en_data.drop(['IPSI_HAKSA_HAKBUN', 'IPSI_YEAR', 'IPSI_JUMIN_NO', 'IPSI_SEX', 'HJBD_ILJA'], axis=1, inplace=True)

In [ ]:
jumin_list = [] 

for i in range(len(en_data)) : 
    if len(str(en_data['HAKJ_JUMIN_NO'][i])) == 6 : 
        year = '19' + str(en_data['HAKJ_JUMIN_NO'][i])
        jumin_list.append(int(year))
    elif len(str(en_data['HAKJ_JUMIN_NO'][i])) == 5 : 
        year = '200' + str(en_data['HAKJ_JUMIN_NO'][i])
        jumin_list.append(int(year))
    elif len(str(en_data['HAKJ_JUMIN_NO'][i])) == 4 : 
        year = '2000' + str(en_data['HAKJ_JUMIN_NO'][i])
        jumin_list.append(int(year)) 
    elif len(str(en_data['HAKJ_JUMIN_NO'][i])) == 3 : 
        year = '20000' + str(en_data['HAKJ_JUMIN_NO'][i])
        jumin_list.append(int(year)) 

In [ ]:
en_data['HAKJ_JUMIN_NO'] = jumin_list
en_data.drop(index=en_data[en_data['HAKJ']=='사망'].index, inplace=True)
en_data = en_data.reset_index(drop=True)

In [ ]:
label_list = []

for i in range(len(en_data)) : 
    if en_data['HAKJ'][i] in ['재학', '복학', '수료', '유급'] : 
        label_list.append(0)
    elif en_data['HAKJ'][i] in ['미복학제적', '자퇴', '미등록제적', '학사제적'] : 
        label_list.append(1)
    elif en_data['HAKJ'][i] == '입대휴학' : 
        if en_data['seq_la'][i] > 4 : #column명 수정
            label_list.append(1)
        else : 
            label_list.append(0)
    elif en_data['HAKJ'][i] in ['일반휴학', '창업휴학', '육아휴학'] : 
        if en_data['seq_la'][i] > 2 : #column명 수정
            label_list.append(1)
        else : label_list.append(0)
    else : 
        label_list.append("NoData")
        
en_data['LABEL'] = label_list

In [ ]:
en_data = en_data[en_data.YYHAKGI > 20202]
en_data = en_data.reset_index(drop=True)

In [ ]:
BS_CNT_list = []

for i in range(len(en_data)) : 
    sum_cnt = 0
    if en_data['HAKJ_BUJ_CODE'][i] != 0 :
        sum_cnt += 1
    if en_data['HAKJ_BS1_CODE'][i] != 0 : 
        sum_cnt += 1
    if en_data['HAKJ_BS2_CODE'][i] != 0 :
        sum_cnt += 1
    if en_data['HAKJ_MACHUMJ_CODE'][i] != 0 : 
        sum_cnt += 1
    BS_CNT_list.append(sum_cnt)
    
en_data['BS_CNT'] = BS_CNT_list

In [ ]:
for i in range(len(en_data)) : 
    if en_data['IPSI_JANG_NAME'][i] == 0 : 
        en_data['IPSI_JANG_NAME'][i] = 0
    else : 
        en_data['IPSI_JANG_NAME'][i] = 1
        
en_data['IPSI_JANG_NAME'] = en_data['IPSI_JANG_NAME'].astype('int64')        

In [ ]:
huhak_list = []
for i in range(len(en_data)) : 
    if 1 in en_data[['H1', 'H2', 'H3', 'H4', 'H5', 'H6']].values[i] : 
        huhak_list.append(1)
    else : 
        huhak_list.append(0)
        
en_data['HUHAK_YN'] = huhak_list

In [ ]:
en_data['IPSI_REPEAT'] = en_data['HAKJ_IPHAK_YY'] - en_data['IPSI_SCHOOL_DATE']

In [ ]:
en_data.drop(['seq', 'seq_la', 'HAKJ', 'IPSI_HOPE_NAME', 'IPSI_DEPT_NAME', 'IPSI_SCHOOL_HAKGWA_NAME', 
              'H1', 'H2', 'H3', 'H4', 'H5', 'H6'], axis=1, inplace=True)

In [ ]:
en_data[['HAKJ_BUJ_CODE', 'HAKJ_BS1_CODE', 'HAKJ_BS2_CODE', 'HAKJ_MACHUMJ_CODE']] = en_data[['HAKJ_BUJ_CODE', 'HAKJ_BS1_CODE', 'HAKJ_BS2_CODE', 'HAKJ_MACHUMJ_CODE']].fillna(0)
en_data[['DAEHAK_NAME', 'HAKGWA_NAME', 'JUNGONG_NAME', 'BUJ_NAME', 'BS1_NAME', 'BS2_NAME', 'BS3_NAME']] = en_data[['DAEHAK_NAME', 'HAKGWA_NAME', 'JUNGONG_NAME', 'BUJ_NAME', 'BS1_NAME', 'BS2_NAME', 'BS3_NAME']].fillna("없음")
en_data[['DAEHAK_NAME', 'HAKGWA_NAME', 'JUNGONG_NAME', 'BUJ_NAME', 'BS1_NAME', 'BS2_NAME', 'BS3_NAME']] = en_data[['DAEHAK_NAME', 'HAKGWA_NAME', 'JUNGONG_NAME', 'BUJ_NAME', 'BS1_NAME', 'BS2_NAME', 'BS3_NAME']].replace('-', '없음')

In [ ]:
data = en_data.copy()

In [ ]:
data['HAKGI']=0
for i in tqdm(range(0,len(data))):
    data['HAKGI'][i] = str(data['YYHAKGI'][i])[:4]

In [ ]:
data['HAKGI'] = data['HAKGI'].astype(int)
new_con = pd.merge(data,census, how='left',left_on=['HAKGI','HAKJ_HAKBUN'], right_on=['YY','HAKBUN'])

In [ ]:
##### 중복되는 열, 전공코드 및 학과 코드 등 삭제 ######
new_con.drop(['DAEHAK_22','JANG_30','JANG_31','GYO_40','GYO_41','DAEHAK_43','DAEHAK_44','TONGHAK_52','TONGHAK_53',\
             'FRESH_58','ETC_59','ETC_60','ETC_61','DAEHAK_62','DAEHAK_63','DAEHAK_64', 'GYO_37', 'GYO_57',
             'ONLINE_38','ONLINE_39','ONLINE_65','ONLINE_66','ONLINE_67','ONLINE_68','ONLINE_69','YY','HAKBUN','HAKGI'],axis=1,inplace=True)

In [ ]:
missing_temp = new_con.loc[new_con['YYHAKGI']==20211]
missing_temp.drop(['DAEHAK_NAME', 'HAKGWA_NAME', 'JUNGONG_NAME', 'BUJ_NAME', 'BS1_NAME', 'BS2_NAME', 'BS3_NAME'], axis=1, inplace=True)
missing_rate = (missing_temp.isnull().sum()/len(missing_temp)).to_dict()

In [ ]:
new_con[['TOTAL_GPA', 'SUIS_CHE_HAKJUM', 'IPSI_OVERLAP_CNT']] = new_con[['TOTAL_GPA', 'SUIS_CHE_HAKJUM', 'IPSI_OVERLAP_CNT']].fillna(0)

In [ ]:
for col in tqdm(new_con.columns):
    if len(new_con[new_con[col].isnull()==True])>=1:
        try:
            new_con[col].fillna(new_con[col].mode()[0],inplace=True)
        except ValueError:
            pass

In [ ]:
train_data = new_con.loc[new_con['YYHAKGI']==20211]
inf_data = new_con.loc[new_con['YYHAKGI']==20222]

In [ ]:
train_data = train_data.loc[train_data['LABEL'] != 'NoData']
inf_data.drop(['LABEL'], axis=1, inplace=True)

In [ ]:
# 0208 추가
inf_data = inf_data.reset_index(drop=True)
for i in range(len(inf_data)) : 
    if inf_data['BD_GBN_NAME'][i] in ['미복학제적', '자퇴', '미등록제적', '학사제적', '수료'] : 
        inf_data.drop([i], axis=0, inplace=True)

In [ ]:
# 0208 추가
train_data['BD_GBN_NAME'].replace({'복학' : '재학',  '유급' : '재학'}, inplace=True)
inf_data['BD_GBN_NAME'].replace({'복학' : '재학', '유급' : '재학'}, inplace=True)

In [ ]:
name_dict = {}
for i in range(len(name)) : 
    name_dict[name['feature'][i]] = name['feature_name'][i]

In [ ]:
# train_data, inference_data 분리, missing_rate pickle 저장
train_data.to_csv('/ml/algorithmlabs.inference/train_data.csv', index=False)
inf_data.to_csv('/ml/algorithmlabs.inference/inference_data.csv', index=False)
joblib.dump(missing_rate, '/ml/algorithmlabs.inference/pickle/missing_rate.pkl')
joblib.dump(name_dict, '/ml/algorithmlabs.inference/pickle/name_dict.pkl')